6/3 (Mon) | Experiment

# Power Analyses of Cohen Kappa Calculation

## 1. Introduction

This notebook conducts power analyses of Cohen Kappa to determine the minimun sample size.
Before starting the evaluation, the following code block loads required packages and define global variables.

In [1]:
from typing import List, Tuple, Dict, Generator, Optional
from pathlib import Path

import numpy as np
import pandas as pd
from jiwer import wer

from utils.cohen_kappa import RCohenKappa

DATA_DIR = Path("/home/matsuura/Development/app/feature_extraction_api/experiment/data")

TASK = ["Arg_Oly", "Cartoon", "RtSwithoutRAA", "RtSwithRAA", "WoZ_Interview"]

FILLER = {"uh", "ah", "um", "mm", "hmm", "oh", "mm-hmm", "er", "mhm", "uh-huh", "er", "erm", "huh", "uhu", "mmhmm", "uhhuh"}

---

## Define Functions

This section defines functions for the power analyses.
The following code block defines functions to load manual csv files in SCTK_inputs and counts required values, such as N_words, N_disfl, and N_pauses.

In [2]:
def remove_filler(df_sctk: pd.DataFrame) -> pd.DataFrame:
    for filler in FILLER:
        mask_filler = (df_sctk["text"] == filler)
        df_sctk = df_sctk[~mask_filler]

    mask_filler = (df_sctk["type"] == "04_filler")
    df_sctk = df_sctk[~mask_filler]

    return df_sctk

def df_sctk_input_loader(task: str) -> Generator[pd.DataFrame, None, None]:
    load_dir = DATA_DIR / f"{task}/10_SCTK_Inputs"
    for manu_csv_path in load_dir.glob("*_manu.csv"):
        df_sctk = pd.read_csv(manu_csv_path)
        yield df_sctk

def count_values(task: str) -> Tuple[int, int, int, int, int]:
    n_files = 0
    n_word = 0
    n_disfl = 0
    n_mcp = 0
    n_ecp = 0

    for df_sctk in df_sctk_input_loader(task):
        mask_disfl = df_sctk["text"] == "<DISFLUENCY>"
        mask_mcp = df_sctk["text"] == "<CI>"
        mask_ecp = df_sctk["text"] == "<CE>"

        mask_tag = (mask_disfl | mask_mcp | mask_ecp)

        n_files += 1
        n_word += len(df_sctk[~mask_tag]) 
        n_disfl += mask_disfl.sum()
        n_mcp += mask_mcp.sum()
        n_ecp += mask_ecp.sum()

    return n_files, n_word, n_disfl, n_mcp, n_ecp

---

## Calculate Proportions

This section calculates the proportions of disfluency words, MCPs, and ECPs.
The following code block counts values for the proportion calculation.

In [3]:
n_files_all = 0
n_word_all = 0
n_disfl_all = 0
n_mcp_all = 0
n_ecp_all = 0

for task in TASK:
    n_files, n_word, n_disfl, n_mcp, n_ecp = count_values(task)

    print(f"[{task}]")
    print(f"\tN files = {n_files}")
    print(f"\tN words = {n_word}")
    print(f"\tN disfl = {n_disfl}")
    print(f"\t N MCPs = {n_mcp}")
    print(f"\t N ECPs = {n_ecp}")
    print("---")

    n_files_all += n_files
    n_word_all += n_word
    n_disfl_all += n_disfl
    n_mcp_all += n_mcp
    n_ecp_all += n_ecp

[Arg_Oly]
	N files = 128
	N words = 16680
	N disfl = 1876
	 N MCPs = 4302
	 N ECPs = 1142
---
[Cartoon]
	N files = 128
	N words = 20958
	N disfl = 2813
	 N MCPs = 5559
	 N ECPs = 1985
---
[RtSwithoutRAA]
	N files = 128
	N words = 21737
	N disfl = 2949
	 N MCPs = 6169
	 N ECPs = 1835
---
[RtSwithRAA]
	N files = 128
	N words = 21829
	N disfl = 2887
	 N MCPs = 6332
	 N ECPs = 1842
---
[WoZ_Interview]
	N files = 1743
	N words = 43467
	N disfl = 3935
	 N MCPs = 7574
	 N ECPs = 2414
---


The following code block calculates the mean number of words for files.

In [4]:
print(f"Mean number of words = {(n_word_all / n_files_all):.03f}")

Mean number of words = 55.286


The following code block calculates the proportions of disfluency words, MCPs, and ECPs.

In [5]:
prop_disfl = n_disfl_all / n_word_all
prop_mcp = n_mcp_all / n_word_all
prop_ecp = n_ecp_all / n_word_all

print(f"Prop disfl = {prop_disfl:.03f}")
print(f"Prop MCPs  = {prop_mcp:.03f}")
print(f"Prop ECPs  = {prop_ecp:.03f}")

Prop disfl = 0.116
Prop MCPs  = 0.240
Prop ECPs  = 0.074


---

## Power Analyses

This section conducts power analyses and estimates the minimum sample size for Cohen Kappa calculation.
The following code block constructs RCohenKappa class for the power analyses.

In [6]:
power_analyzer = RCohenKappa(debug=False)

The following code block conducts a power analysis for Cohen Kappa of disfluency word detection.

In [7]:
n_disfl = power_analyzer.power_analysis(
    kappa_null=0.3, kappa_alt=0.61, 
    props=prop_disfl,
    alpha=0.05, beta=0.8
)

print(f"Minimum sample size for disfl. detection = {n_disfl:.03f}")

Minimum sample size for disfl. detection = 150.226


The following code block conducts power analyses for Cohen Kappa of MCP & ECP classification.

In [8]:
n_pause = power_analyzer.power_analysis(
    kappa_null=0.3, kappa_alt=0.61, 
    props=[prop_mcp, prop_ecp, 1 - prop_mcp - prop_ecp],
    alpha=0.05, beta=0.8
)

print(f"Minimum sample size for pause location classification = {n_pause:.03f}")

Minimum sample size for pause location classification = 66.433
